In [1]:
import numpy as np
import keras
import os
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error
import pickle

import pandas as pd
import re
from itertools import chain
import os
# from param_lstm_model import LSTM_model
from pandas import Series
from keras.preprocessing.text import Tokenizer
import numpy as np
from numpy import subtract
import sys
import traceback
import pickle

Using TensorFlow backend.


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import tensorflow as tf

LIMIT = 12
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*LIMIT)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')

In [3]:
import pandas as pd

df_sys_train = pd.read_csv('df_sys_train.csv')
df_sys_test = pd.read_csv('df_sys_test.csv')

In [4]:
# 执行路径异常检测
print("执行路径异常检测")
from execution_path_detect import execution_path
execution_path(df_sys_train, df_sys_test)

执行路径异常检测
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

the length of yhat is 642
the length of anomaly_sequence [49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 230, 231, 23

In [5]:
df_anomaly_message = pd.read_csv('execute_path_anormal.csv')
df_anomaly_message.head()

,LineId,timestamp,Content,line,spell_templates,EventTemplate,EventId,param,match,time_gap,ParameterList
0,59,1.618970e+09,sysmonitor[39896]|bash[12078](parent:bash[1205...,2021-04-21T02:00:26.099055+08:00|sysmonitor[39...,{'lcsseq': 'sysmonitor[<:NUM:>]|bash[<:NUM:>](...,sysmonitor[<:NUM:>]|bash[<:NUM:>](parent:bash[...,4,['sysmonitor[<:NUM:>]|bash[<:NUM:>](parent:bas...,1,0,[['sysmonitor[<:NUM:>]|bash[<:NUM:>](parent:ba...
1,60,1.618970e+09,sysmonitor[39896]|bash[12078](parent:bash[1205...,2021-04-21T02:00:26.099229+08:00|sysmonitor[39...,{'lcsseq': 'sysmonitor[<:NUM:>]|bash[<:NUM:>](...,sysmonitor[<:NUM:>]|bash[<:NUM:>](parent:bash[...,4,['sysmonitor[<:NUM:>]|bash[<:NUM:>](parent:bas...,1,0,[['sysmonitor[<:NUM:>]|bash[<:NUM:>](parent:ba...
2,61,1.618970e+09,sysmonitor[39896]|bash[12078](parent:bash[1205...,2021-04-21T02:00:26.205224+08:00|sysmonitor[39...,{'lcsseq': 'sysmonitor[<:NUM:>]|bash[<:NUM:>](...,sysmonitor[<:NUM:>]|bash[<:NUM:>](parent:bash[...,4,['sysmonitor[<:NUM:>]|bash[<:NUM:>](parent:bas...,1,0,[['sysmonitor[<:NUM:>]|bash[<:NUM:>](parent:ba...
3,62,1.618970e+09,sysmonitor[39896]|bash[12078](parent:bash[1205...,2021-04-21T02:00:26.205377+08:00|sysmonitor[39...,{'lcsseq': 'sysmonitor[<:NUM:>]|bash[<:NUM:>](...,sysmonitor[<:NUM:>]|bash[<:NUM:>](parent:bash[...,4,['sysmonitor[<:NUM:>]|bash[<:NUM:>](parent:bas...,1,0,[['sysmonitor[<:NUM:>]|bash[<:NUM:>](parent:ba...
4,63,1.618970e+09,sysmonitor[39896]|bash[12078](parent:bash[1205...,2021-04-21T02:00:26.275918+08:00|sysmonitor[39...,{'lcsseq': 'sysmonitor[<:NUM:>]|bash[<:NUM:>](...,sysmonitor[<:NUM:>]|bash[<:NUM:>](parent:bash[...,4,['sysmonitor[<:NUM:>]|bash[<:NUM:>](parent:bas...,1,0,[['sysmonitor[<:NUM:>]|bash[<:NUM:>](parent:ba...


In [6]:
df_anomaly_message.shape

(550, 11)

In [7]:
df_anomaly_message.Content.unique()

array(['sysmonitor[39896]|bash[12078](parent:bash[12050]) send SIGKILL to bash[37635]. <-[bash /opt/fusionstorage/agent//script/dsware_agent_(12050)]<-[./dsware_agent(39347)]',
       'sysmonitor[39896]|bash[12078](parent:bash[12050]) send SIGKILL to free[37646]. <-[bash /opt/fusionstorage/agent//script/dsware_agent_(12050)]<-[./dsware_agent(39347)]',
       'sysmonitor[39896]|bash[12078](parent:bash[12050]) send SIGKILL to bash[37727]. <-[bash /opt/fusionstorage/agent//script/dsware_agent_(12050)]<-[./dsware_agent(39347)]',
       'sysmonitor[39896]|bash[12078](parent:bash[12050]) send SIGKILL to iostat[37738]. <-[bash /opt/fusionstorage/agent//script/dsware_agent_(12050)]<-[./dsware_agent(39347)]',
       'sysmonitor[39896]|bash[12078](parent:bash[12050]) send SIGKILL to bash[37995]. <-[bash /opt/fusionstorage/agent//script/dsware_agent_(12050)]<-[./dsware_agent(39347)]',
       'sysmonitor[39896]|bash[12078](parent:bash[12050]) send SIGKILL to iostat[38022]. <-[bash /opt/fusionstora